In [ ]:
!pip install evaluate Pillow==9.0.0
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorboard-2.12.1-py3-none-any.whl (5.6 MB)
  Using cached py7zr-0.20.4-py3-none-any.whl (66 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 40.7 MB/s eta 0:00:00
  Using cached multivolumefile-0.2.3-py3-none-any.whl (17 kB)
  Usin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
DATAPATH = "drive/MyDrive/CS4248 Team 19/translated_data/final_random_2k_de.csv"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_id = "google/flan-t5-large" # ryan: change to flan-t5-large
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model_id = "google/flan-t5-large" # ryan: change to flan-t5-large later
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda:0")

In [ ]:
import pandas as pd

random_de_df = pd.read_csv(DATAPATH).values.tolist()

In [ ]:
print (random_de_df[0][0])
print (random_de_df[0][1])
print (random_de_df[0][2])
print (random_de_df[0][3])
print (random_de_df[0][4])
print (random_de_df[0][5])

0
Eine Person mit braunen Haaren trägt ein blaues Hemd und lila Bandanna um ihren Hals mischt ein Design in eine runde, Ziegelsäule.
Die Person, die ein blaues Hemd trägt, meißelt auf einer Ziegelsäule.
entailment
Eine Person meißelt ein Design aus Ziegelsteinen.
german


In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix

updated_prompts_de = []
for record in random_de_df:
    prem = record[1]
    hypo = record[2]
    text = f"Premise: {prem} Hypothesis: {hypo} Please provide a justification for whether the hypothesis is entailed by the premise, contradicted by the premise, or if it cannot be determined."
    updated_prompts_de.append(text)

In [ ]:
from pprint import pprint

print (updated_prompts_de[0])
print (updated_prompts_de[1])

Premise: Eine Person mit braunen Haaren trägt ein blaues Hemd und lila Bandanna um ihren Hals mischt ein Design in eine runde, Ziegelsäule. Hypothesis: Die Person, die ein blaues Hemd trägt, meißelt auf einer Ziegelsäule. Please provide a justification for whether the hypothesis is entailed by the premise, contradicted by the premise, or if it cannot be determined.
Premise: Zwei Männer sitzen in einem Wohnzimmer, einer trägt kulturelle Kleidung und ein anderer in einem weißen Hemd und Shorts, betrachten auf ihrem Fahrrad und Moped. Hypothesis: Zwei Männer in blauen Anzügen verkaufen Fahrräder und Mopeds. Please provide a justification for whether the hypothesis is entailed by the premise, contradicted by the premise, or if it cannot be determined.


In [ ]:
BS = 50
device = torch.device("cuda:0")

mapper = {"neutral": 0, "contradiction": 1, "entailment": 2}
y_test = [mapper[record[3]] for record in random_de_df]

all_pred = []
count = 0

# for i in range(0, len(updated_prompts_de), BS):
batch = updated_prompts_de[:50]
inputs = tokenizer(batch, return_tensors="pt", padding=True)

outputs = model.generate(
                inputs['input_ids'].to(device),
                attention_mask=inputs['attention_mask'].to(device),
                min_length=1,
                max_new_tokens=40,
                num_beams=4
            )
preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for pred in preds:
    if count < 50:
        all_pred.append(pred)
        count += 1
    else:
        break

# print (len(y_pred))
print (len(y_test))
print (len(all_pred))

2000
50


In [ ]:
# cls_data = list(zip(y_test, y_pred))       
# cls_data = pd.DataFrame(cls_data, columns=["Actual", "Prediction"])
# conf_matrix = confusion_matrix(y_test, y_pred)
# cls_report = classification_report(y_test, y_pred)

In [ ]:
print (len(all_pred))

to_save = []

limit = len(all_pred)
for item in zip(random_de_df[:limit], all_pred):
    idx, prem, hypo, label, expl, _ = item[0] # _ is the language [german]
    pred = item[1]

    to_save.append([
        prem,
        hypo,
        label,
        expl,
        pred
    ])

50


In [ ]:
pprint (to_save[:5])

[['Eine Person mit braunen Haaren trägt ein blaues Hemd und lila Bandanna um '
  'ihren Hals mischt ein Design in eine runde, Ziegelsäule.',
  'Die Person, die ein blaues Hemd trägt, meißelt auf einer Ziegelsäule.',
  'entailment',
  'Eine Person meißelt ein Design aus Ziegelsteinen.',
  'A person with braunen hairs carries a blaues Hemd and lila Bandanna um '
  'herhals mischt a design in a round,'],
 ['Zwei Männer sitzen in einem Wohnzimmer, einer trägt kulturelle Kleidung und '
  'ein anderer in einem weißen Hemd und Shorts, betrachten auf ihrem Fahrrad '
  'und Moped.',
  'Zwei Männer in blauen Anzügen verkaufen Fahrräder und Mopeds.',
  'contradiction',
  'Kulturelle Kleidung und ein weißes Hemd und Shorts sind informelle, lässige '
  'Kleidung, während blaue Anzüge sind geschäftliche Kleidung und deutlich '
  'anders in der Art. Auch sitzen und überlegen auf einem Fahrrad und Moped '
  'würde implizieren, dass sie nicht versuchen, sie zu verkaufen.',
  "The two men can't be in a 

In [ ]:
to_save = pd.DataFrame(to_save, columns=["premise", "hypothesis", "label", "explanation", "pred_explanation"])
to_save.to_csv(f"drive/MyDrive/CS4248 Team 19/Findings/flan-t5-large-ots-german-eval-first50.csv")